source:  
https://github.com/roatienza/Deep-Learning-Experiments/blob/master/Experiments/Tensorflow/GAN/dcgan_mnist.py  
https://zhuanlan.zhihu.com/p/22386494 (介紹1)  
https://zhuanlan.zhihu.com/p/22389906 (介紹2)  

In [ ]:
'''
DCGAN on MNIST using Keras
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
Dependencies: tensorflow 1.0 and keras 2.0
Usage: python3 dcgan_mnist.py
'''

import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt

class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

class DCGAN(object):
    def __init__(self, img_rows=28, img_cols=28, channel=1):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        self.G.summary()
        return self.G

    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM

class MNIST_DCGAN(object):
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channel = 1

        self.x_train = input_data.read_data_sets("mnist",\
        	one_hot=True).train.images
        self.x_train = self.x_train.reshape(-1, self.img_rows,\
        	self.img_cols, 1).astype(np.float32)

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            if save_interval>0:
                if (i+1)%save_interval==0:
                    self.plot_images(save2file=True, samples=noise_input.shape[0],\
                        noise=noise_input, step=(i+1))

    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = 'mnist.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "mnist_%d.png" % step
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

if __name__ == '__main__':
    mnist_dcgan = MNIST_DCGAN()
    timer = ElapsedTimer()
    mnist_dcgan.train(train_steps=10000, batch_size=256, save_interval=500)
    timer.elapsed_time()
    mnist_dcgan.plot_images(fake=True)
    mnist_dcgan.plot_images(fake=False, save2file=True)

/home/tony/dnn/anaconda2/envs/myjupyter/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None

41: [D loss: 1.120831, acc: 0.501953]  [A loss: 6.569053, acc: 0.000000]
42: [D loss: 0.474764, acc: 0.779297]  [A loss: 0.685098, acc: 0.570312]
43: [D loss: 1.142588, acc: 0.500000]  [A loss: 6.378450, acc: 0.000000]
44: [D loss: 0.460514, acc: 0.789062]  [A loss: 0.641844, acc: 0.621094]
45: [D loss: 1.061949, acc: 0.501953]  [A loss: 5.958043, acc: 0.000000]
46: [D loss: 0.425565, acc: 0.824219]  [A loss: 0.820232, acc: 0.429688]
47: [D loss: 1.011864, acc: 0.501953]  [A loss: 6.051521, acc: 0.000000]
48: [D loss: 0.405273, acc: 0.837891]  [A loss: 0.727746, acc: 0.519531]
49: [D loss: 1.096560, acc: 0.500000]  [A loss: 6.932978, acc: 0.000000]
50: [D loss: 0.494640, acc: 0.744141]  [A loss: 0.440544, acc: 0.859375]
51: [D loss: 1.110068, acc: 0.500000]  [A loss: 5.813478, acc: 0.000000]
52: [D loss: 0.376688, acc: 0.892578]  [A loss: 0.911562, acc: 0.324219]
53: [D loss: 0.960966, acc: 0.503906]  [A loss: 6.330828, acc: 0.000000]
54: [D loss: 0.391854, acc: 0.867188]  [A loss: 0.7

153: [D loss: 0.524962, acc: 0.673828]  [A loss: 2.289649, acc: 0.003906]
154: [D loss: 0.818959, acc: 0.509766]  [A loss: 6.428046, acc: 0.000000]
155: [D loss: 0.740503, acc: 0.683594]  [A loss: 0.010168, acc: 1.000000]
156: [D loss: 2.211154, acc: 0.500000]  [A loss: 0.883324, acc: 0.382812]
157: [D loss: 0.951178, acc: 0.513672]  [A loss: 4.422016, acc: 0.000000]
158: [D loss: 0.507112, acc: 0.779297]  [A loss: 0.239832, acc: 0.984375]
159: [D loss: 1.291469, acc: 0.500000]  [A loss: 4.644156, acc: 0.000000]
160: [D loss: 0.551421, acc: 0.755859]  [A loss: 0.129374, acc: 0.996094]
161: [D loss: 1.368887, acc: 0.500000]  [A loss: 3.572902, acc: 0.000000]
162: [D loss: 0.504605, acc: 0.734375]  [A loss: 0.878125, acc: 0.402344]
163: [D loss: 1.239998, acc: 0.500000]  [A loss: 7.240736, acc: 0.000000]
164: [D loss: 0.970139, acc: 0.617188]  [A loss: 0.007669, acc: 1.000000]
165: [D loss: 2.294699, acc: 0.500000]  [A loss: 0.499786, acc: 0.804688]
166: [D loss: 1.023519, acc: 0.503906]

264: [D loss: 0.837902, acc: 0.599609]  [A loss: 0.069182, acc: 1.000000]
265: [D loss: 1.853369, acc: 0.500000]  [A loss: 2.891843, acc: 0.000000]
266: [D loss: 0.691845, acc: 0.595703]  [A loss: 1.137747, acc: 0.230469]
267: [D loss: 1.180488, acc: 0.490234]  [A loss: 5.085429, acc: 0.000000]
268: [D loss: 0.744695, acc: 0.599609]  [A loss: 0.208914, acc: 0.968750]
269: [D loss: 1.518743, acc: 0.498047]  [A loss: 4.449958, acc: 0.000000]
270: [D loss: 0.789261, acc: 0.572266]  [A loss: 0.161880, acc: 0.996094]
271: [D loss: 1.606058, acc: 0.500000]  [A loss: 4.178552, acc: 0.000000]
272: [D loss: 0.738714, acc: 0.583984]  [A loss: 0.282517, acc: 0.929688]
273: [D loss: 1.456501, acc: 0.496094]  [A loss: 4.500454, acc: 0.000000]
274: [D loss: 0.723868, acc: 0.619141]  [A loss: 0.172728, acc: 0.992188]
275: [D loss: 1.354809, acc: 0.501953]  [A loss: 3.755240, acc: 0.000000]
276: [D loss: 0.717195, acc: 0.607422]  [A loss: 0.297775, acc: 0.933594]
277: [D loss: 1.415162, acc: 0.496094]

...

819: [D loss: 2.326306, acc: 0.527344]  [A loss: 11.147060, acc: 0.000000]
820: [D loss: 1.855155, acc: 0.531250]  [A loss: 0.013173, acc: 1.000000]
821: [D loss: 4.532968, acc: 0.500000]  [A loss: 2.085907, acc: 0.226562]
822: [D loss: 1.890219, acc: 0.562500]  [A loss: 8.575846, acc: 0.000000]
823: [D loss: 1.193246, acc: 0.560547]  [A loss: 0.132038, acc: 0.945312]
824: [D loss: 2.786444, acc: 0.505859]  [A loss: 6.254716, acc: 0.000000]
825: [D loss: 0.958779, acc: 0.587891]  [A loss: 1.381450, acc: 0.347656]
826: [D loss: 1.865381, acc: 0.535156]  [A loss: 7.554563, acc: 0.000000]
827: [D loss: 1.142860, acc: 0.535156]  [A loss: 0.323638, acc: 0.839844]
828: [D loss: 2.258297, acc: 0.521484]  [A loss: 7.717062, acc: 0.000000]
829: [D loss: 0.982936, acc: 0.576172]  [A loss: 0.289197, acc: 0.855469]
830: [D loss: 1.973074, acc: 0.523438]  [A loss: 6.615812, acc: 0.000000]
831: [D loss: 0.943112, acc: 0.599609]  [A loss: 0.793180, acc: 0.566406]
832: [D loss: 1.713440, acc: 0.542969

930: [D loss: 1.036324, acc: 0.564453]  [A loss: 0.448800, acc: 0.726562]
931: [D loss: 1.985589, acc: 0.552734]  [A loss: 6.479195, acc: 0.000000]
932: [D loss: 0.896530, acc: 0.607422]  [A loss: 0.285927, acc: 0.851562]
933: [D loss: 1.808544, acc: 0.529297]  [A loss: 4.876399, acc: 0.000000]
934: [D loss: 0.891609, acc: 0.609375]  [A loss: 1.812593, acc: 0.191406]
935: [D loss: 1.524228, acc: 0.582031]  [A loss: 7.740957, acc: 0.000000]
936: [D loss: 1.108836, acc: 0.560547]  [A loss: 0.095227, acc: 0.992188]
937: [D loss: 2.737018, acc: 0.503906]  [A loss: 4.873147, acc: 0.000000]
938: [D loss: 0.989916, acc: 0.623047]  [A loss: 3.032412, acc: 0.042969]
939: [D loss: 1.382207, acc: 0.580078]  [A loss: 7.181217, acc: 0.000000]
940: [D loss: 1.124617, acc: 0.552734]  [A loss: 0.156713, acc: 0.957031]
941: [D loss: 2.683642, acc: 0.507812]  [A loss: 6.023719, acc: 0.000000]
942: [D loss: 0.953864, acc: 0.587891]  [A loss: 0.315940, acc: 0.859375]
943: [D loss: 1.898682, acc: 0.541016]

1041: [D loss: 0.916361, acc: 0.603516]  [A loss: 0.198753, acc: 0.917969]
1042: [D loss: 2.038127, acc: 0.513672]  [A loss: 6.850323, acc: 0.000000]
1043: [D loss: 0.934400, acc: 0.597656]  [A loss: 0.804969, acc: 0.554688]
1044: [D loss: 1.765260, acc: 0.525391]  [A loss: 8.706205, acc: 0.000000]
1045: [D loss: 1.262280, acc: 0.541016]  [A loss: 0.013284, acc: 1.000000]
1046: [D loss: 3.870612, acc: 0.500000]  [A loss: 3.804511, acc: 0.015625]
1047: [D loss: 1.340340, acc: 0.544922]  [A loss: 6.559784, acc: 0.000000]
1048: [D loss: 0.821704, acc: 0.613281]  [A loss: 0.734787, acc: 0.605469]
1049: [D loss: 1.683161, acc: 0.535156]  [A loss: 9.885137, acc: 0.000000]
1050: [D loss: 1.346102, acc: 0.562500]  [A loss: 0.005870, acc: 1.000000]
1051: [D loss: 4.614882, acc: 0.500000]  [A loss: 3.585448, acc: 0.046875]
1052: [D loss: 1.501546, acc: 0.552734]  [A loss: 7.893253, acc: 0.000000]
1053: [D loss: 0.993029, acc: 0.587891]  [A loss: 0.240877, acc: 0.890625]
1054: [D loss: 2.182309, 

1151: [D loss: 2.233718, acc: 0.511719]  [A loss: 7.757558, acc: 0.000000]
1152: [D loss: 1.016592, acc: 0.593750]  [A loss: 0.125542, acc: 0.953125]
1153: [D loss: 2.273234, acc: 0.511719]  [A loss: 6.127300, acc: 0.000000]
1154: [D loss: 0.928764, acc: 0.621094]  [A loss: 2.802919, acc: 0.121094]
1155: [D loss: 1.625575, acc: 0.541016]  [A loss: 8.014194, acc: 0.000000]
1156: [D loss: 0.972985, acc: 0.580078]  [A loss: 0.281992, acc: 0.875000]
1157: [D loss: 2.105134, acc: 0.529297]  [A loss: 9.997329, acc: 0.000000]
1158: [D loss: 1.511948, acc: 0.554688]  [A loss: 0.003556, acc: 1.000000]
1159: [D loss: 5.109355, acc: 0.500000]  [A loss: 3.291367, acc: 0.070312]
1160: [D loss: 1.825828, acc: 0.513672]  [A loss: 9.127437, acc: 0.000000]
1161: [D loss: 1.145949, acc: 0.564453]  [A loss: 0.148093, acc: 0.929688]
1162: [D loss: 2.543301, acc: 0.503906]  [A loss: 8.292441, acc: 0.000000]
1163: [D loss: 1.059429, acc: 0.587891]  [A loss: 0.599464, acc: 0.664062]
1164: [D loss: 1.991670, 

1260: [D loss: 7.237757, acc: 0.501953]  [A loss: 3.361272, acc: 0.429688]
1261: [D loss: 7.086642, acc: 0.501953]  [A loss: 4.044038, acc: 0.390625]
1262: [D loss: 7.249311, acc: 0.500000]  [A loss: 2.921367, acc: 0.539062]
1263: [D loss: 7.232054, acc: 0.494141]  [A loss: 10.151714, acc: 0.074219]
1264: [D loss: 6.763124, acc: 0.486328]  [A loss: 16.091528, acc: 0.000000]
1265: [D loss: 8.059052, acc: 0.500000]  [A loss: 16.106422, acc: 0.000000]
1266: [D loss: 8.050802, acc: 0.500000]  [A loss: 16.083206, acc: 0.000000]
1267: [D loss: 8.049097, acc: 0.500000]  [A loss: 16.059258, acc: 0.000000]
1268: [D loss: 8.047041, acc: 0.500000]  [A loss: 16.047302, acc: 0.000000]
1269: [D loss: 8.031905, acc: 0.500000]  [A loss: 15.969795, acc: 0.000000]
1270: [D loss: 8.029876, acc: 0.500000]  [A loss: 15.225248, acc: 0.011719]
1271: [D loss: 6.253259, acc: 0.515625]  [A loss: 0.000006, acc: 1.000000]
1272: [D loss: 7.938692, acc: 0.500000]  [A loss: 0.000001, acc: 1.000000]
1273: [D loss: 7.